#Librairie 

In [0]:
!pip install langchain
!pip install tiktoken
!pip install openpyxl
!pip install -qU langchain-community langchain-openai
!pip install langchain-openai pydantic openpyxl
!pip install pypdf
!pip install -qU langchain-community langchain-openai
!pip install pdf2image Pillow
!pip install pdfplumber
!pip install tqdm
!pip install unstructured

dbutils.library.restartPython()

Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
Note: you may need to restart the kernel using dbutils.library.r

#Début divers 

In [0]:
import pandas as pd
import os
import json
from pydantic import BaseModel, Field
from langchain_core.messages import HumanMessage, SystemMessage
import asyncio
from glob import glob
import os
import requests
from typing import Dict, Optional, Any, Union, List
from dataclasses import dataclass
import base64

In [0]:
@dataclass
class AuthConfig:
    base_url: str
    client_id: str
    client_secret: str
    scope: str

class AuthClient:
    def __init__(self, config: AuthConfig):
        self.config = config

    def get_token(self) -> Dict[str, str]:
        """Get OAuth2 access token."""
        try:
            response = requests.post(
                f"{self.config.base_url}/as/token.oauth2",
                headers={
                    "Content-Type": "application/x-www-form-urlencoded"
                },
                data={
                    "grant_type": "client_credentials",
                    "client_id": self.config.client_id,
                    "client_secret": self.config.client_secret,
                    "scope": self.config.scope
                }
            )
            response.raise_for_status()
            return response.json()
            
        except requests.exceptions.RequestException as e:
            raise Exception(f"Authentication failed: {str(e)}")

# Usage example
def get_auth_token() -> str:
    config = AuthConfig(
        base_url="https://onelogin.stg.axa.com",
        client_id="VJKkswZb",
        client_secret="yG3pl6nkYhvJlQVD1y7xbZBl692YRytdpUJeODKg5BAutoEgxhkqzXLFYGqH5zlp",
        scope="urn:grp:chatgpt"
    )
    
    client = AuthClient(config)
    result = client.get_token()
    return result.get("access_token", "")

class SecureGPTChat:
   def __init__(self, 
                token: str,
                temperature: float = 0.05,
                base_url: str = "https://api-pp.se.axa-go.axa.com/ago-m365-securegpt-bapi-v1-vrs",
                model: str = "gpt-4o-2024-08-06"):
       self.token = token
       self.temperature = temperature
       self.base_url = base_url
       self.model = model
       self.costs = {
           'input_token': 2.50/1_000_000,  # Cost per token
           'output_token': 10.00/1_000_000,
           'base_image_cost': 0.001913      # Base cost for 1024x1024
       }

   def calculate_cost(self, response, image_path=None):
       total_cost = 0
       metrics = {'token_cost': 0, 'image_cost': 0}

       if 'usage' in response:
           input_tokens = response['usage'].get('prompt_tokens', 0)
           output_tokens = response['usage'].get('completion_tokens', 0)
           metrics['token_cost'] = (input_tokens * self.costs['input_token'] + 
                                  output_tokens * self.costs['output_token'])
           total_cost += metrics['token_cost']

       if image_path:
           from PIL import Image
           with Image.open(image_path) as img:
               width, height = img.size
               pixels = width * height
               metrics['image_cost'] = (pixels / (1024 * 1024)) * self.costs['base_image_cost']
               total_cost += metrics['image_cost']

       metrics['total_cost'] = total_cost
       return metrics

   def encode_image(self, image_path: str) -> str:
       if image_path.startswith('data:image'):
           return image_path.split('base64,')[1]
       with open(image_path, "rb") as image_file:
           return base64.b64encode(image_file.read()).decode('utf-8')

   def prepare_messages(self, messages: List[Dict[str, Any]], image_path: Optional[str] = None) -> List[Dict]:
       formatted_messages = []
       for msg in messages:
           content = msg["content"]
           if image_path and msg == messages[-1]:
               content = [
                   {"type": "image_url", 
                    "image_url": f"data:image/jpeg;base64,{self.encode_image(image_path)}",
                    "detail": "auto"},
                   {"type": "text", "text": content}
               ]
           formatted_messages.append({
               "role": msg.get("role", "user"),
               "content": content
           })
       return formatted_messages

   def chat(self, messages: List[Dict[str, Any]], image_path: Optional[str] = None, **kwargs) -> Dict:
       url = f"{self.base_url}/deployments/{self.model}/chat/completions"
       headers = {
           "Content-Type": "application/json",
           "Authorization": f"Bearer {self.token}"
       }
       data = {
           "messages": self.prepare_messages(messages, image_path),
           "temperature": self.temperature,
           "max_tokens": kwargs.get("max_tokens", 4000)
       }

       try:
           response = requests.post(url, headers=headers, json=data)
           response.raise_for_status()
           api_response = response.json()
           cost_metrics = self.calculate_cost(api_response, image_path)
           api_response['cost_metrics'] = cost_metrics
           return api_response
       except requests.exceptions.RequestException as e:
           print(f"API Error Details: {response.text if 'response' in locals() else 'No response'}")
           raise Exception(f"API call failed: {str(e)}")

   async def ainvoke(self, messages):
       formatted_messages = []
       image_path = None
       
       for msg in messages:
           if isinstance(msg.content, list):
               for item in msg.content:
                   if item.get('type') == 'image_url':
                       image_path = item['image_url']['url']
                   elif item.get('type') == 'text':
                       formatted_messages.append({
                           "role": "user" if msg.type == "human" else "system",
                           "content": item.get('text', '')
                       })
           else:
               formatted_messages.append({
                   "role": "user" if msg.type == "human" else "system",
                   "content": msg.content
               })

       response = self.chat(formatted_messages, image_path=image_path) if image_path else self.chat(formatted_messages)
       return type('Response', (), {'content': response['choices'][0]['message']['content']})()

model = SecureGPTChat(
    temperature=0.0,
    model="gpt-4o-2024-08-06",
    token=get_auth_token()  # Your existing token retrieval method
)

###chat tools

In [0]:
def create_chat_interface():
   token = get_auth_token()
   model = SecureGPTChat(
        temperature=0.0,
        model="gpt-4o-mini-2024-07-18",
        token=get_auth_token()  # Your existing token retrieval method
    )
   history = []
   
   def chat_with_model():
       while True:
           user_input = input("\nVous: ")
           if user_input.lower() in ['quit', 'exit', 'bye']:
               print("\nAu revoir!")
               break
               
           history.append({"role": "user", "content": user_input})
           response = model.chat(history)
           bot_response = response['choices'][0]['message']['content']
           history.append({"role": "assistant", "content": bot_response})
           print("\nAssistant:", bot_response)
   
   return chat_with_model
# Utilisation
# chat = create_chat_interface()
# chat()

#create json structure from image excel

In [0]:
import os

def delete_png_files():
    for file_name in os.listdir('.'):
        if file_name.endswith('.png'):
            os.remove(file_name)

delete_png_files()

In [0]:
# Open Excel file and access corresponding page
import openpyxl
workbook = openpyxl.load_workbook('input_referentials.xlsx')
print(workbook.sheetnames)

['fields', 'other', 'endoso_80', 'endoso_157', 'endoso_97', 'endoso_38', 'endoso_132', 'endoso_67', 'endoso_65', 'endoso_69', 'endoso_153', 'endoso_135', 'endoso_82', 'endoso_122', 'endoso_150', 'endoso_158', 'endoso_78', 'endoso_127', 'endoso_36', 'endoso_155', 'endoso_27', 'endoso_142', 'endoso_154', 'endoso_84', 'endoso_73', 'endoso_131', 'endoso_74']


In [0]:
import openpyxl
import os
import matplotlib.pyplot as plt
import json
from pathlib import Path
from tqdm import tqdm

def create_output_directories():
    """Crée les répertoires nécessaires pour les outputs"""
    Path('temp_images').mkdir(exist_ok=True)
    Path('json_structures').mkdir(exist_ok=True)

def get_all_sheet_names(excel_file: str) -> list:
    """Récupère tous les noms d'onglets de l'Excel"""
    wb = openpyxl.load_workbook(excel_file)
    return wb.sheetnames

def excel_sheet_to_image(excel_file: str, sheet_name: str, output_path: str):
    """Convertit un onglet Excel en image"""
    wb = openpyxl.load_workbook(excel_file)
    ws = wb[sheet_name]
    
    # Convertir les données en liste pour matplotlib
    data = []
    for row in ws.rows:
        data.append([cell.value if cell.value is not None else '' for cell in row])
    
    # Créer la figure
    fig, ax = plt.subplots(figsize=(30, 15))
    ax.axis('off')
    
    # Créer le tableau
    table = ax.table(cellText=data,
                    cellLoc='left',
                    loc='center',
                    edges='BRTL',
                    bbox=[0, 0, 1, 1])
    
    # Formater
    table.auto_set_font_size(False)
    table.set_fontsize(10)
    table.auto_set_column_width(col=list(range(len(data[0]))))
    
    # Sauvegarder
    plt.savefig(output_path, dpi=300, bbox_inches='tight', pad_inches=0)
    plt.close()

def extract_structure_with_securegpt(model, image_path: str) -> dict:
    """Extrait la structure JSON via SecureGPT"""
    with open('output_referential_example.json', 'r') as f:
        example_json = json.load(f)

    
    messages = [
        {"role": "system", "content": f"""Tu es un expert en assurance, en contrat d'assurance  en analyse de tableaux Excel contenant des paramètres d'endoso.
        Ta tâche est d'extraire la structure exacte du tableau en JSON : sachant que c'est une image issu d'un fichier excel du coup si tu vois par exemple des trous dans le tableau que t'observes c'est juste parce que c'est retraité et donc normalement les cellules sont censés être fusionnés
        - Identifie les paramètres standards et d'exception
        - Conserve la hiérarchie exacte du tableau
        - Fais attention aux mots qui peuvent servir à rien genre le titre etc
        - Les données que tu vois sont une structure d'endoso (option de contrat facultatif mexicain) 
        - Inclus tous les détails pertinents (noms, valeurs, types)

        De plus je te fournis ce json en exemple pour que tu puisses t'en inspirer : \n
        {json.dumps(example_json, indent=2)}
\n
        Retourne UNIQUEMENT les données au format  JSON, sans texte explicatif."""},
        {"role": "user", "content": "Analyse cette image et extrais sa structure en JSON"}
    ]
    
    try:
        response = model.chat(messages, image_path=image_path)
        # Nettoyer la réponse
        json_str = response['choices'][0]['message']['content']
        json_str = json_str.replace('```json', '').replace('```', '').strip()
        return json.loads(json_str)
    except Exception as e:
        print(f"Erreur lors de l'extraction: {str(e)}")
        return {}

def main(excel_file: str, model):
    """Fonction principale"""
    # Création des dossiers
    create_output_directories()
    
    # Récupération des onglets
    sheet_names = get_all_sheet_names(excel_file)
    print(f"Nombre d'onglets trouvés: {len(sheet_names)}")
    
    # Traitement de chaque onglet
    for sheet_name in tqdm(sheet_names, desc="Traitement des onglets"):
        if 'endoso' in sheet_name:
            print(sheet_name)
            try:
                # Génération de l'image
                image_path = f"temp_images/{sheet_name}.png"
                excel_sheet_to_image(excel_file, sheet_name, image_path)
                
                # Extraction de la structure
                structure = extract_structure_with_securegpt(model, image_path)
                
                # Sauvegarde du JSON
                json_path = f"json_structures/{sheet_name}.json"
                with open(json_path, 'w', encoding='utf-8') as f:
                    json.dump(structure, f, indent=2, ensure_ascii=False)
                    
            except Exception as e:
                print(f"Erreur lors du traitement de l'onglet {sheet_name}: {str(e)}")
                continue
            
model = SecureGPTChat(
    temperature=0.0,
    model="gpt-4o-2024-08-06",
    token=get_auth_token()
)

# Lancement du traitement
main('input_referentials.xlsx', model)

Nombre d'onglets trouvés: 27


Traitement des onglets:   0%|          | 0/27 [00:00<?, ?it/s]

endoso_80


Traitement des onglets:  11%|█         | 3/27 [00:15<02:00,  5.04s/it]

endoso_157


...

Traitement des onglets:  19%|█▊        | 5/27 [00:38<03:11,  8.71s/it]

endoso_38


Traitement des onglets:  22%|██▏       | 6/27 [01:03<04:45, 13.60s/it]

endoso_132


com.databricks.backend.common.rpc.CommandCancelledException
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$5(SequenceExecutionState.scala:136)
	at scala.Option.getOrElse(Option.scala:189)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3(SequenceExecutionState.scala:136)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3$adapted(SequenceExecutionState.scala:133)
	at scala.collection.immutable.Range.foreach(Range.scala:158)
	at com.databricks.spark.chauffeur.SequenceExecutionState.cancel(SequenceExecutionState.scala:133)
	at com.databricks.spark.chauffeur.ExecContextState.cancelRunningSequence(ExecContextState.scala:730)
	at com.databricks.spark.chauffeur.ExecContextState.$anonfun$cancel$1(ExecContextState.scala:448)
	at scala.Option.getOrElse(Option.scala:189)
	at com.databricks.spark.chauffeur.ExecContextState.cancel(ExecContextState.scala:448)
	at com.databricks.spark.chauffeur.ChauffeurState.cancelExecutio

In [0]:
with open('json_structures/endoso_38.json', 'r') as f:
        example_json = json.load(f)

example_json

{'endoso_details': {'parameters': {'column_parameters': [{'column_parameter_name': 'Pago directo',
     'column_parameter_details': [{'sub_column_parameter_name': '',
       'editable': True,
       'active': True,
       'type': 'std',
       'row_parameters': [{'row_parameter_name': 'Suma Asegurada',
         'editable': True,
         'active': True,
         'type': 'std',
         'value_type': 'digits/currency (pesos, UMAM)/de la poliza',
         'unit': ''},
        {'row_parameter_name': 'Deducible',
         'editable': True,
         'active': True,
         'type': 'std',
         'value_type': 'digits/de la poliza',
         'unit': ''},
        {'row_parameter_name': 'Coaseguro',
         'editable': True,
         'active': True,
         'type': 'std',
         'value_type': 'digits',
         'unit': ''},
        {'row_parameter_name': 'Tope de Coaseguro',
         'editable': True,
         'active': True,
         'type': 'std',
         'value_type': 'digits',
     

#Endossos extractor 

In [0]:
import pandas as pd
import json
from pathlib import Path
from tqdm import tqdm

class EndosoParameterExtractor:
    def __init__(self, model, structure_dir: str = "json_structures"):
        self.model = model
        self.structure_dir = structure_dir
        self.structures_cache = {}
        
        # Charger les exemples à l'initialisation
        with open('input_endoso_example.txt', 'r') as f:
            self.input_example = f.read()
        with open('output_endoso_example.json', 'r') as f:
            self.output_example = json.load(f)
    
    def load_structure(self, endoso_code: str) -> dict:
        if endoso_code in self.structures_cache:
            return self.structures_cache[endoso_code]
        
        json_path = Path(self.structure_dir) / f"{endoso_code}.json"
        try:
            with open(json_path, 'r', encoding='utf-8') as f:
                structure = json.load(f)
                self.structures_cache[endoso_code] = structure
                return structure
        except Exception as e:
            print(f"Erreur lors du chargement de la structure pour {endoso_code}: {str(e)}")
            return {}
    
    def extract_parameters(self, text: str, structure: dict,endoso_code: str) -> dict:
        
        conversation = [
            {"role": "system", "content": "Tu es un extracteur intelligent de garanties d'endosos. Tu sortiras UNIQUEMENT LE JSON PAS DE PHRASE A COTE QUAND JE TE LE DEMANDERAI UNIQUEMENT LE JSON"},
            {"role": "user", "content": "Je vais te montrer comment extraire les garanties."},
            {"role": "user", "content": f"Input exemple:\n{self.input_example}"},
            {"role": "user", "content": f"Output exemple:\n{json.dumps(self.output_example, indent=2)}"},
            {"role": "assistant", "content": "Compris, je vais adapter l'extraction au format demandé en fonction de l'endoso avec tous les paramètres comme l'input exemple vers l'output exemple."},
            {"role": "user", "content": f"""Voici la structure de l'endoso, il faudra bien respecter la structure et les données de l'endoso le texte que je te fournis :\n{json.dumps(structure, indent=2)}
                        
            Le json fournit donne uniquement la structure et les modalités possible par contre TRES IMPORTANT SUIS SURTOUT L'EXEMPLE FOURNI PLUS HAUT EN RESPECTANT le data model du  json

            De plus il faudra (c'est super important) trouver les paramètres de chaque endoso : {{'row_parameter_name': 'deducible', 'value': '100', 'unit': 'USD', 'type': 'std'}} un exemple de paramètre.

            Utilise cette structure et le texte suivant pour générer le JSON final:\n{text}

            Tu transformeras donc le texte en json structurée en t'inspirant des exemples et en appliquant bien évidemment la structure demandé complété des paramètres demandés qui se trouvent dans le texte, en gros les deux avec toutes les informations sois exhaustives. S'il y a un paramètre que tu n'arrives pas à trouver dans le texte tu mettras "ATTENTION PARAMETRE MANQUANT" dans le json à la place du paramètre.

            Toujours dans un format Json. Jamais de données superflus après pas de phrases rien. 

            l'endoso_master_id ici est : {endoso_code.replace('endoo_','')}, l'endoso est : {endoso_code}

            Si tu ne trouves rien renvoie un json vide. 
            """}
        ]
        try:
            response = self.model.chat(conversation)
            json_str = response['choices'][0]['message']['content'].replace('```json','').replace('```','').strip()
            return {
                'parameters': json.loads(json_str),
                'cost_metrics': response.get('cost_metrics', {})
            }
        except Exception as e:
            print(f"Erreur lors de l'extraction des paramètres: {str(e)}")
            return {'parameters': {}, 'cost_metrics': {}}

def process_single_example(df: pd.DataFrame, model):
    """Traite un seul exemple aléatoire pour test"""
    # Sélection aléatoire
    random_row = df.sample(n=1).iloc[0]
    
    # Récupérer le code endoso et le texte
    endoso_code = random_row['endoso_code']  # Assurez-vous que cette colonne existe
    text_content = random_row['CONT']        # Assurez-vous que cette colonne existe
    
    print(f"Code endoso sélectionné : {endoso_code}")
    
    # Initialiser l'extracteur
    extractor = EndosoParameterExtractor(model)
    
    # Charger la structure
    structure = extractor.load_structure(endoso_code)
    if not structure:
        print("Structure non trouvée")
        return
        
    # Extraire les paramètres
    result = extractor.extract_parameters(text_content, structure,endoso_code)
    
    # Afficher le résultat
    print("\nRésultat de l'extraction :")
    print(json.dumps(result['parameters'], indent=2, ensure_ascii=False))
    
    return result

def process_dataframe(df: pd.DataFrame, model, output_file: str = 'results.json'):
    """Traite l'ensemble du DataFrame"""
    extractor = EndosoParameterExtractor(model)
    results = []
    
    for _, row in tqdm(df.iterrows(), total=len(df), desc="Traitement des endosos"):
        try:
            # S'assurer que nous avons accès aux bonnes colonnes
            endoso_code = row['endoso_code']  # Assurez-vous que cette colonne existe
            text_content = row['CONT']        # Assurez-vous que cette colonne existe
            
            structure = extractor.load_structure(endoso_code)
            
            if not structure:
                print(f"Structure non trouvée pour {endoso_code}")
                continue
            
            result = extractor.extract_parameters(text_content, structure,endoso_code)
            
            results.append({
                'endoso_code': endoso_code,
                'text': text_content,
                'extracted_parameters': result['parameters'],
                'cost_metrics': result['cost_metrics']
            })
            
        except Exception as e:
            print(f"Erreur lors du traitement de la ligne : {str(e)}")
            continue
    
    with open(output_file, 'w', encoding='utf-8') as f:
        json.dump(results, f, indent=2, ensure_ascii=False)
    
    return results

In [0]:
model = SecureGPTChat(
    temperature=0.0,
    model="gpt-4o-mini-2024-07-18",
    token=get_auth_token()
)

# Charger le DataFrame
df = pd.read_csv('dataframe_migration.csv')

# Charger le mapping et ajouter la colonne endoso_code
with open('mapping_endosoName_code.json', 'r') as f:
    mapping = json.load(f)

def get_endoso_code(description: str, mapping: list) -> str:
    description_lower = description.lower().strip()
    for item in mapping:
        for key in item.keys():
            if key.lower() in description_lower or description_lower in key.lower():
                return item[key]
    return None

# Ajouter la colonne endoso_code
df['endoso_code'] = df['DRIPCION'].apply(lambda x: get_endoso_code(x, mapping))

In [0]:
results = process_dataframe(df,model)

Traitement des endosos:  98%|█████████▊| 49/50 [07:20<00:05,  5.27s/it]

Erreur lors du chargement de la structure pour exclusion instituciones medica: [Errno 2] No such file or directory: 'json_structures/exclusion instituciones medica.json'
Structure non trouvée pour exclusion instituciones medica


Traitement des endosos: 100%|██████████| 50/50 [07:28<00:00,  8.96s/it]


[{'endoso_code': 'endoso_36',
  'text': 'La Cobertura de Urgencias  Médicas  en  el  Extranjero   se  regirA  de  acuerdo a la siguiente definiciOn:   Para efectos de esta clAusula se entenderA como Urgencia  Médica en  el  Extranjero  cuando  el asegurado o alguno de  sus dependientes  estando  de visita en otro paIs  presente en forma sUbita e imprevista cualquier  alteraciOn de su estado de salud  sea por  accidente o  enfermedad  que  ponga en peligro su vida o sus funciones  corporales y que requiera  de  tratamiento médico y/o quirUrgico    Deducible y Coaseguro   Esta cobertura es adicional a la cobertura  bAsica  aplicAndose el  be-  neficio mAximo y  deducible que se establece en la  carAtula  de la pO-  liza   En esta cobertura no se aplicarA coaseguro    TerminaciOn de la Urgencia Médica    Una vez tratado el paciente y  estabilizada su condiciOn  patolOgica  y  si de acuerdo a la opiniOn del médico tratante  el paciente  puede con-  tinuar el tratamiento fuera del hospital 